In [6]:
import pandas as pd
import requests
import io
import numpy
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LogisticRegression
import random

url = "https://raw.githubusercontent.com/the-data-science-union/DSU-W21-Personality/main/cleaned_youngpeople_data.csv" # Make sure the url is the raw version of the file on GitHub
download = requests.get(url).content
personality_questions = pd.read_csv(io.StringIO(download.decode('utf-8')))

personality_questions['Country'] = round((personality_questions['Country'] + personality_questions['Folk']) / 2)
personality_questions['Rock'] = round((personality_questions['Rock'] + personality_questions['Rock n roll']+personality_questions['Alternative'] + personality_questions['Punk'] + personality_questions['Metal or Hardrock']) / 5)
personality_questions['Classical music'] = round((personality_questions['Classical music'] + personality_questions['Opera'] + personality_questions['Swing, Jazz']) / 3)
personality_questions['Dance'] = round((personality_questions['Dance'] + personality_questions['Techno, Trance']) / 2)
personality_questions['Top Genre'] = personality_questions[['Dance', 'Country', 'Classical music', 'Pop', 
                                                            'Rock', 'Hiphop, Rap', 'Latino']].idxmax(axis=1)

print(personality_questions.groupby("Top Genre").count())
personality_questions=personality_questions.drop(columns=['Music', 'Slow songs or fast songs', 'Folk',
       'Musical', 'Pop', 'Metal or Hardrock',
       'Punk', 'Hiphop, Rap', 'Reggae, Ska', 'Swing, Jazz', 'Rock n roll',
       'Latino', 'Techno, Trance', 'Opera'])

personality_questions= personality_questions.drop(columns=['Dance', 'Country', 'Classical music','Rock', 
                                                           'Alternative', 'Village - town', 'House - block of flats', 
                                                           'Left - right handed', 'Education', 'Gender', 'Lying', 
                                                           'Punctuality', 'Only child', 'Internet usage', 'Smoking', 'Alcohol'])

X=personality_questions.drop(columns=['Top Genre'])
Y=personality_questions[['Top Genre']]
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)
X_train = X_train.to_numpy()
y_train = y_train.to_numpy()
X_test = X_test.to_numpy()
y_test = y_test.to_numpy()
clf = LogisticRegression(random_state=0).fit(X_train, y_train)

model = SelectFromModel(clf, prefit=True,max_features=10)
print(model.get_support(indices=True))
print(personality_questions.columns[model.get_support(indices=True)])
X_new = model.transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_new, personality_questions[['Top Genre']], test_size=0.2)

y_train = y_train.to_numpy()
y_test = y_test.to_numpy()
clf = LogisticRegression(random_state=0).fit(X_train, y_train)

print(clf.score(X_test, y_test))
predictions = clf.predict(X_test)
unique, counts = numpy.unique(predictions, return_counts=True)
print(numpy.asarray((unique, counts)).T)
print(clf.score(X_test, y_test))

                 Music  ...  House - block of flats
Top Genre               ...                        
Classical music     96  ...                      96
Country             64  ...                      64
Dance              180  ...                     180
Hiphop, Rap        117  ...                     117
Latino              71  ...                      71
Pop                327  ...                     327
Rock               122  ...                     122

[7 rows x 150 columns]
[ 5 11 22 28 31 32 40 42 67 82]
Index(['Sci-fi', 'Action', 'Reading', 'Art exhibitions', 'Dancing',
       'Musical instruments', 'Theatre', 'Adrenaline sports',
       'Criminal damage', 'God'],
      dtype='object')


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example

0.4336734693877551
[['Classical music' 19]
 ['Country' 5]
 ['Dance' 21]
 ['Hiphop, Rap' 8]
 ['Latino' 2]
 ['Pop' 131]
 ['Rock' 10]]
0.4336734693877551


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
